In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir analysis
!cp -r '/content/drive/MyDrive/Open vs proprietary ICU datasets' /content
%cd '/content/Open vs proprietary ICU datasets'
!pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1t907vtyppuUe9raIUtcXIt6-CLnnAxuo/Open vs proprietary ICU datasets
/content/drive/.shortcut-targets-by-id/1t907vtyppuUe9raIUtcXIt6-CLnnAxuo/Open vs proprietary ICU datasets


# 0. Prepare for environment and dataset

## 0.1 import packages

In [2]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install country_converter

In [3]:
# !pip install pycountry

In [4]:
import pandas as pd

In [5]:
# import pandas as pd
# import fuzzywuzzy
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# import ast
# import country_converter as coco
# import pycountry

## 0.2 read data ( Please check the data path)

In [6]:
Control_pubs =  pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control/control_group_pubs.csv")
MIMIC_pubs = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC/mimic_pubs.csv")
Biobank_pubs = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank/biobank_pubs.csv")

In [7]:
MIMIC_author = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC/mimic_auths.csv")
Biobank_author = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank/biobank_auths.csv")
Control_author = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control/control_group_auths.csv")

In [8]:
JournalIF = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/Journal_Impact_Factor/MergedJournalImpactFactorUnique3.csv")
CountryClass=pd.read_excel("/content/Open vs proprietary ICU datasets/Datasets/CountryClass/CountryClassWordBank.xlsx")

In [9]:
MIMIC_affli = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC/mimic_affils.csv")
Biobank_affli = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank/biobank_affils.csv")
Control_affli = pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control/control_group_affils.csv")

In [10]:
Control_affli.columns

Index(['Unnamed: 0', 'aff_city', 'aff_city_id', 'aff_country',
       'aff_country_code', 'aff_id', 'aff_name', 'aff_raw_affiliation',
       'aff_state', 'aff_state_code', 'pub_id', 'researcher_id', 'first_name',
       'last_name', 'year', 'gender', 'gender_ints', 'income_class',
       'income_class_num'],
      dtype='object')

In [11]:
print(len(MIMIC_pubs))
print(len(Control_pubs))
print(len(MIMIC_author))
print(len(Control_author))
print(len(MIMIC_affli))
print(len(Control_affli))

2307
2912
13879
20539
13967
25676


In [12]:
Control_affli.columns

Index(['Unnamed: 0', 'aff_city', 'aff_city_id', 'aff_country',
       'aff_country_code', 'aff_id', 'aff_name', 'aff_raw_affiliation',
       'aff_state', 'aff_state_code', 'pub_id', 'researcher_id', 'first_name',
       'last_name', 'year', 'gender', 'gender_ints', 'income_class',
       'income_class_num'],
      dtype='object')

In [ ]:
concept_df = pd.read_csv('/content/Open vs proprietary ICU datasets/Dimensions Results/Concept Scores for topics within "Health & Medicine"/concept_scores_dentistry.csv')

In [ ]:
concept_df.head(5)

## 0.3 preprocess data

In [ ]:
JournalIF['Journal name'] = JournalIF['Journal name'].fillna("NAN")
for i in range(len(JournalIF)):
  x = JournalIF['Journal name'].iloc[i]
  # print(x)
  JournalIF['Journal name'].iloc[i] = JournalIF['Journal name'].iloc[i].upper()
# JournalIF
JournalIF[JournalIF['Journal name']=='NAN']

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Unnamed: 0,Journal name,JCR Abbreviation,ISSN,eISSN,Category,Total Citations,2021 JIF,JIF Quartile,2021 JCI,% of OA Gold,5 Year JIF,AIS Quartile,JCI Percentile


In [ ]:
MIMIC_test = MIMIC_pubs.copy()


# Compare_test['journal.title'] = Compare_test['journal.title'].fillna('NAN')
# for i in range(len(Compare_test)):
#   x = Compare_test['journal.title'].iloc[i]
#   # print(x)
#   Compare_test['journal.title'].iloc[i] = Compare_test['journal.title'].iloc[i].upper()



In [ ]:
MIMIC_test['journal.title'].iloc[1].upper()

'INTERNATIONAL JOURNAL OF CLINICAL PRACTICE'

In [ ]:
JournalIF.columns

Index(['Unnamed: 0', 'Journal name', 'JCR Abbreviation', 'ISSN', 'eISSN',
       'Category', 'Total Citations', '2021 JIF', 'JIF Quartile', '2021 JCI',
       '% of OA Gold', '5 Year JIF', 'AIS Quartile', 'JCI Percentile'],
      dtype='object')

# 1. Match Impact Factor

## 1.1 FuzzyMatch

In [ ]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [ ]:
def fuzzyMatch(publication_df,JournalIF):
  JIF_names =list(JournalIF[JournalIF['Journal name'].notna()==True]['Journal name'].astype("string").unique())
  publication_Journal_list = list(publication_df[publication_df['journal.title'].notna()==True]['journal.title'].astype("string").unique())
  print(publication_Journal_list)
  publication_df['5_Year_JIF'] = 'NAN'
  publication_df['2021_JIF'] = 'NAN'
  publication_df['ISSN'] = 'NAN'
  publication_df['Category'] = 'NAN' # Category from Claritive Journal IF
  # for i in range(len(publication_df)):
  for i in range(len(publication_df)):
    x = publication_df['journal.title'].iloc[i].upper()
    # print('x',x)
    match = match_names(x, JIF_names, 85) # you can change the match rate here
    # print(i)
    try:
      # match = match_names(x, JIF_names, 85) # you can change the match rate here
      # print(match[1])

      if match[1]>= 85:
        JournalName = str(match[0])
        print("journalname",JournalName)
        publication_df['5_Year_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['5 Year JIF'].iloc[0]
        print("check",JournalIF[JournalIF['Journal name']== JournalName]['5 Year JIF'].iloc[0])
        publication_df['2021_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['2021 JIF'].iloc[0]
        publication_df['ISSN'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['ISSN'].iloc[0]
        publication_df['Category'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['Category'].iloc[0]
      else:
        publication_df['5_Year_JIF'].iloc[i] = "did not match criteria"
        publication_df['2021_JIF'].iloc[i] = "did not match criteria"
        publication_df['ISSN'].iloc[i] = "did not match criteria"
        publication_df['Category'].iloc[i] = "did not match criteria"
        # print("else")

    except:
      publication_df['5_Year_JIF'].iloc[i] = "math error"
      publication_df['2021_JIF'].iloc[i] = "match error"
      publication_df['ISSN'].iloc[i] = "match error"
      publication_df['Category'].iloc[i] = "match error"
      # print("except")
      # print(match)
  return publication_df

In [ ]:
# testFuzzyMatch = fuzzyMatch(Compare_copy,JournalIF)

NameError: ignored

In [ ]:
# Compare.columns

In [ ]:
# def fuzzyMatch(publication_df,JournalIF):
#   JIF_names =list(JournalIF[JournalIF['Journal name'].notna()==True]['Journal name'].astype("string").unique())
#   publication_Journal_list = list(publication_df[publication_df['journal.title'].notna()==True]['journal.title'].astype("string").unique())
#   print(publication_Journal_list)
#   publication_df['5_Year_JIF'] = 'NAN'
#   publication_df['2021_JIF'] = 'NAN'
#   publication_df['ISSN'] = 'NAN'
#   publication_df['Category'] = 'NAN' # Category from Claritive Journal IF
#   # for i in range(len(publication_df)):
#   for i in range(len(publication_df)):
#     x = publication_df['journal.title'].iloc[i].upper()
#     # print('x',x)
#     match = match_names(x, JIF_names, 85) # you can change the match rate here
#     # print(i)
#     try:
#       # match = match_names(x, JIF_names, 85) # you can change the match rate here
#       # print(match[1])

#       if match[1]>= 85:
#         JournalName = str(match[0])
#         print("journalname",JournalName)
#         publication_df['5_Year_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['5 Year JIF'].iloc[0]
#         print("check",JournalIF[JournalIF['Journal name']== JournalName]['5 Year JIF'].iloc[0])
#         publication_df['2021_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['2021 JIF'].iloc[0]
#         publication_df['ISSN'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['ISSN'].iloc[0]
#         publication_df['Category'].iloc[i] = JournalIF[JournalIF['Journal name']== JournalName]['Category'].iloc[0]
#       else:
#         publication_df['5_Year_JIF'].iloc[i] = "did not match criteria"
#         publication_df['2021_JIF'].iloc[i] = "did not match criteria"
#         publication_df['ISSN'].iloc[i] = "did not match criteria"
#         publication_df['Category'].iloc[i] = "did not match criteria"
#         # print("else")

#     except:
#       publication_df['5_Year_JIF'].iloc[i] = "math error"
#       publication_df['2021_JIF'].iloc[i] = "match error"
#       publication_df['ISSN'].iloc[i] = "match error"
#       publication_df['Category'].iloc[i] = "match error"
#       # print("except")
#       # print(match)
#   return publication_df

In [ ]:
# testFuzzyMatch = fuzzyMatch(Compare_copy,JournalIF)

In [ ]:
# testFuzzyMatch.to_csv("/content/Open vs proprietary ICU datasets/Dimensions Results/New Merged Files (Use these)/compare_publications_addIF.csv")

In [ ]:
# testFuzzyMatch[testFuzzyMatch['5_Year_JIF']=='did not match criteria']['journal.title']

In [ ]:
# notmatch = testFuzzyMatch[(testFuzzyMatch['5_Year_JIF']=='did not match criteria')&(testFuzzyMatch['journal.title']!='NAN')][['journal.title','5_Year_JIF']]

In [ ]:
# notmatch['journal.title'].unique()

In [ ]:
# notmatch

## 1.2 ExactMatch

In [ ]:
# def ExactMatchIF(publication_df,JournalIF):
#   # JournalIF['Journal name'] = JournalIF['Journal name'].fillna("NAN")
#   publication_df['5_Year_JIF'] = 'NAN'
#   publication_df['2021_JIF'] = 'NAN'
#   publication_df['ISSN'] = 'NAN'
#   publication_df['Category'] = 'NAN'


#   publication_df['journal.title'] = publication_df['journal.title'].fillna('NAN')
#   for i in range(len(publication_df)):
#   # for i in range(10):
#     print(i)
#     Journal_name = publication_df['journal.title'].iloc[i].upper()
#     print(Journal_name)
#     try:
#       publication_df['5_Year_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']==Journal_name]['5 Year JIF'].iloc[0]
#       publication_df['2021_JIF'].iloc[i] = JournalIF[JournalIF['Journal name']==Journal_name]['2021_JIF'].iloc[0]
#       publication_df['ISSN'].iloc[i] = JournalIF[JournalIF['Journal name']==Journal_name]['ISSN'].iloc[0]
#       publication_df['Category'].iloc[i] = JournalIF[JournalIF['Journal name']==Journal_name]['Category'].iloc[0] ## incase there's duplicate records

#     except:
#       print("didn't match")
#       publication_df['5_Year_JIF'].iloc[i] ='NAN'
#       publication_df['2021_JIF'].iloc[i] = 'NAN'
#       publication_df['ISSN'].iloc[i] = 'NAN'
#       publication_df['Category'].iloc[i] = 'NAN'
#   return publication_df


In [ ]:
# Compare_copy = Compare.copy()
# test_ExactMatchIF = ExactMatchIF(Compare_copy,JournalIF)

# 2. Country Income Match

In [ ]:
# import country_convert package as coco , import ast to load string to dictionary
def addCountryIncomeClass(author_df):
  author_df['Country'] = 'NAN'
  author_df['CountryIncomeClass'] = 'NAN'
  author_df['Affliation_id'] = 'NAN'
  for i in range(len(author_df)):
    # print(i)
    try:
      toDict = ast.literal_eval(author_df.iloc[i].affiliations)

      country_ISO2 = toDict[0].get('country_code')
      # print(country_ISO2)
      country_ISO3 = coco.convert(names=country_ISO2, to='ISO3')
      # print("ISO3 ",country_ISO3)
      # print("type",type(country_ISO3))

      IncomeClass = CountryClass[CountryClass.Code==country_ISO3]['Income group'].to_string(index = False)
      # print(IncomeClass)

      author_df['Country'].iloc[i] = toDict[0].get('country')
      author_df['Affliation_id'].iloc[i] = toDict[0].get('id')
      author_df['CountryIncomeClass'].iloc[i] = IncomeClass
      # print(author_df['CountryIncomeClass'].iloc[i])
    except:
      author_df['CountryIncomeClass'].iloc[i] = 'NAN'
      # print('match error')
  return author_df

In [ ]:
MIMIC_author_addIC = addCountryIncomeClass(MIMIC_author)
# Biobank_author_addIC = addCountryIncomeClass(Biobank_author)
Control_author_addIC = addCountryIncomeClass(Control_author)

Streaming output truncated to the last 5000 lines.
<ipython-input-18-71ff2f289159>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_df['Country'].iloc[i] = toDict[0].get('country')
<ipython-input-18-71ff2f289159>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_df['Affliation_id'].iloc[i] = toDict[0].get('id')
<ipython-input-18-71ff2f289159>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_df['CountryIncomeClass

In [ ]:
MIMIC_affli.columns

Index(['Unnamed: 0', 'aff_city', 'aff_city_id', 'aff_country',
       'aff_country_code', 'aff_id', 'aff_name', 'aff_raw_affiliation',
       'aff_state', 'aff_state_code', 'pub_id', 'researcher_id', 'first_name',
       'last_name', 'year', 'gender', 'gender_ints', 'income_class',
       'income_class_num'],
      dtype='object')

In [ ]:
MIMIC_affli.tail()

,Unnamed: 0,aff_city,aff_city_id,aff_country,aff_country_code,aff_id,aff_name,aff_raw_affiliation,aff_state,aff_state_code,pub_id,researcher_id,first_name,last_name,year,gender,gender_ints,income_class,income_class_num
13962,13962,Borstel,2945988.0,Germany,DE,grid.418187.3,Research Center Borstel - Leibniz Lung Center,"Division of Structural Biochemistry, Departmen...",NaN,NaN,pub.1008209435,ur.012014630262.69,Anthony P.,Moran,2010,male,0.0,High income,4.0
13963,13963,Fort Collins,5577147.0,United States,US,grid.47894.36,Colorado State University,"Department of Microbiology, Immunology and Pat...",Colorado,US-CO,pub.1008209435,ur.012014630262.69,Anthony P.,Moran,2010,male,0.0,High income,4.0
13964,13964,Brisbane,2174003.0,Australia,AU,grid.1022.1,Griffith University,"Institute for Glycomics, Gold Coast Campus, Gr...",Queensland,AU-QLD,pub.1008209435,ur.012014630262.69,Anthony P.,Moran,2010,male,0.0,High income,4.0
13965,13965,Galway,2964180.0,Ireland,IE,grid.6142.1,"National University of Ireland, Galway","School of Natural Sciences, National Universit...",NaN,NaN,pub.1008209435,ur.012014630262.69,Anthony P.,Moran,2010,male,0.0,High income,4.0
13966,13966,Brisbane,2174003.0,Australia,AU,grid.1022.1,Griffith University,"School of Natural Sciences, National Universit...",Queensland,AU-QLD,pub.1008209435,ur.012014630262.69,Anthony P.,Moran,2010,male,0.0,High income,4.0


In [ ]:
MIMIC_author.researcher_id

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4                       NaN
                ...        
13833     ur.01251520113.88
13834      ur.0767456033.83
13835    ur.013705635017.95
13836      ur.0743377731.49
13837    ur.012014630262.69
Name: researcher_id, Length: 13838, dtype: object

In [ ]:
# write the df to folder
MIMIC_author_addIC.to_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC_author_addIC_v3.csv")
# Biobank_author_addIC.to_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank_author_addIC_v3.csv")
Control_author_addIC.to_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control_author_addIC_v3.csv")

In [ ]:
# import country_convert package as coco , import ast to load string to dictionary
def addCountryIncomeClasstoAffli(row):
  country_code = row['aff_country_code']

  if pd.isna(country_code): return "NoCountryCode"

  try:
    country_ISO2 = country_code
    # print(country_ISO2)
    country_ISO3 = coco.convert(names=country_ISO2, to='ISO3')

    IncomeClass = CountryClass[CountryClass.Code==country_ISO3]['Income group'].to_string(index = False)
  except:
    IncomeClass = 'NotMatched'
    # print('match error')
  return IncomeClass

In [ ]:
Control_affli['CountryIncomeClass']= Control_affli.apply(addCountryIncomeClasstoAffli, axis=1)
MIMIC_affli['CountryIncomeClass'] = MIMIC_affli.apply(addCountryIncomeClasstoAffli, axis = 1)
Biobank_affli['CountryIncomeClass'] = Biobank_affli.apply(addCountryIncomeClasstoAffli, axis = 1)

In [ ]:
MIMIC_affli_addIC

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-166e5985f167>", line 1, in <module>
    MIMIC_affli_addIC
NameError: name 'MIMIC_affli_addIC' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultra

NameError: ignored

In [ ]:
 CountryClass[CountryClass.Code == 'RE']

In [ ]:
Control_affli.columns

## Use pycountry_ Auth_affli_addIC

In [ ]:
import numpy as np

missing_codes = []
dataset_missing_country = []
def getcountryclass(row):
  aff_country_code = row['aff_country_code']
  if pd.isna(aff_country_code):
    # print(row)
    return str('NoCountryCode')

  if aff_country_code == 'XK':
    country_ISO3 =coco.convert(names = 'XK', to = 'ISO3') ## use coco.covert package for edge case in pycountry packages

  else:
    try:
      country_ISO3 = pycountry.countries.get(alpha_2 = aff_country_code).alpha_3
    except:
      country_ISO3 = 'NoISO3'
      missing_codes.append(aff_country_code)

  #print("country_ISO3", country_ISO3)
  if country_ISO3 is None:
    missing_codes.append(aff_country_code)
    return np.nan
  else:
    try:
       IncomeClass = CountryClass[CountryClass.Code == country_ISO3]['Income group'].values[0]
       #print("IncomeClass",IncomeClass)
       return IncomeClass
    except:
      missing_codes.append(aff_country_code)
      return 'NotMatched'


In [ ]:
len(MIMIC_affli)

13967

In [ ]:
Control_affli['income_class'] = Control_affli.apply(getcountryclass, axis=1)
MIMIC_affli['income_class'] = MIMIC_affli.apply(getcountryclass, axis=1)
Biobank_affli['income_class'] = Biobank_affli.apply(getcountryclass, axis = 1)

KeyboardInterrupt: ignored

In [ ]:
Control_author['income_class'] = Control_author.apply(getcountryclass, axis=1)
MIMIC_author['income_class'] = MIMIC_author.apply(getcountryclass, axis=1)
Biobank_author['income_class'] = Biobank_author.apply(getcountryclass, axis = 1)

In [ ]:
len(Control_affli[Control_affli.income_class == 'NoCountryCode'])

853

### Add Year Match

In [ ]:
# Add year match
# year is year matched from publication datasets
Control_affli['year'] = 0
for i in range(len(Control_affli)):
  Control_affli['year'].iloc[i]=Control_pubs[Control_pubs['id']==Control_affli['pub_id'].iloc[i]].year


# Biobank_affli['year'] = 0
# for i in range(len(Biobank_affli)):
#   Biobank_affli['year'].iloc[i]=Biobank_pubs[Biobank_pubs['id']==Biobank_affli['pub_id'].iloc[i]].year

<ipython-input-16-36e5c1235c78>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Control_affli['year'].iloc[i]=Control_pubs[Control_pubs['id']==Control_affli['pub_id'].iloc[i]].year
<ipython-input-16-36e5c1235c78>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Biobank_affli['year'].iloc[i]=Biobank_pubs[Biobank_pubs['id']==Biobank_affli['pub_id'].iloc[i]].year


In [ ]:
MIMIC_affli['year'] = 0
for i in range(len(MIMIC_affli)):
  thisYear = MIMIC_pubs[MIMIC_pubs['id']==MIMIC_affli['pub_id'].iloc[i]].year
  try:
    MIMIC_affli['year'].iloc[i]=int(thisYear)
  except:
    print(i)
    print(MIMIC_pubs[MIMIC_pubs['id']==MIMIC_affli['pub_id'].iloc[i]].year)
    MIMIC_affli['year'].iloc[i] = 'NaN'


<ipython-input-17-13a11b4c8ef2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MIMIC_affli['year'].iloc[i]=int(thisYear)


In [ ]:
MIMIC_affli.year.value_counts()

2022    7249
2021    3707
2020    1350
2011     567
2019     445
2018     235
2017     127
2016     104
2015      79
2013      42
2014      23
2010      21
2012      18
Name: year, dtype: int64

### Add LMIC match

In [ ]:
def is_lmic(income_class):
    if income_class in ['Lower middle income', 'Low income']:
        return 'Yes'
    else:
        return 'No'
MIMIC_affli['LMIC'] = MIMIC_affli['income_class'].apply(is_lmic)
# Biobank_affli['LMIC'] =Biobank_affli['income_class'].apply(is_lmic)
Control_affli['LMIC'] = Control_affli['income_class'].apply(is_lmic)

In [ ]:
MIMIC_affli['LMIC'].value_counts()

No     13312
Yes      655
Name: LMIC, dtype: int64

Add First/Last Author

In [ ]:
# Initialize the 'Author_Order' column with 'Middle'
MIMIC_affli['Author_Order'] = 'Middle'

# Get the researcher_id for the first author for each publication
first_author_ids = MIMIC_affli.groupby('pub_id').first()['researcher_id']

# Get the researcher_id for the last author for each publication
last_author_ids = MIMIC_affli.groupby('pub_id').last()['researcher_id']

# Mark the first authors in MIMIC_affli based on their researcher_id
MIMIC_affli.loc[MIMIC_affli['researcher_id'].isin(first_author_ids), 'Author_Order'] = 'First'

# Mark the last authors in MIMIC_affli based on their researcher_id
MIMIC_affli.loc[MIMIC_affli['researcher_id'].isin(last_author_ids), 'Author_Order'] = 'Last'

# Initialize the 'Author_Order' column with 'Middle'
Control_affli['Author_Order'] = 'Middle'

# Get the researcher_id for the first author for each publication
first_author_ids = Control_affli.groupby('pub_id').first()['researcher_id']

# Get the researcher_id for the last author for each publication
last_author_ids = Control_affli.groupby('pub_id').last()['researcher_id']

# Mark the first authors in Control_affli based on their researcher_id
Control_affli.loc[Control_affli['researcher_id'].isin(first_author_ids), 'Author_Order'] = 'First'

# Mark the last authors in Control_affli based on their researcher_id
Control_affli.loc[Control_affli['researcher_id'].isin(last_author_ids), 'Author_Order'] = 'Last'


####  Missingness analysis

In [ ]:
MIMIC_affli.to_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC_author_affiliations_addIC_v3.csv")
# Biobank_affli.to_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank_author_affliations_addIC_v3.csv")
Control_affli.to_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/control_author_affiliations_addIC_v3.csv")

In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1t907vtyppuUe9raIUtcXIt6-CLnnAxuo/Open vs proprietary ICU datasets'

# Analysis and Plots

In [ ]:
import pandas as pd
MIMIC_author= pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC_author_addIC_v3.csv")

MIMIC_auth= pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC_author_addIC_v3.csv")
MIMIC_affli = pd.read_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/MIMIC_author_affiliations_addIC_v3.csv")
# Biobank_affli = pd.read_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/Biobank_author_affliations_addIC_v3.csv")
Control_author= pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control_author_addIC_v3.csv")
Control_auth= pd.read_csv("/content/Open vs proprietary ICU datasets/Datasets/2023Data/Control_author_addIC_v3.csv")
Control_affli = pd.read_csv("/content/drive/MyDrive/Open vs proprietary ICU datasets/Datasets/2023Data/control_author_affiliations_addIC_v3.csv")

In [ ]:
print(len(MIMIC_affli))
print(len(Control_affli))
# print(len(MIMIC_pubs))
# print(len(MIMIC_author))
print(MIMIC_affli.columns)

15639
25676
Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'aff_city', 'aff_city_id',
       'aff_country', 'aff_country_code', 'aff_id', 'aff_name',
       'aff_raw_affiliation', 'aff_state', 'aff_state_code', 'pub_id',
       'researcher_id', 'first_name', 'last_name', 'year', 'gender',
       'gender_ints', 'income_class', 'income_class_num',
       'missing_affiliation', 'LMIC', 'full_name', 'Author_Order'],
      dtype='object')


## Count Authors


In [ ]:
# MIMIC_affli.head(5)
# Create a new 'full_name' column that combines the 'first_name' and 'last_name' columns
Control_affli['full_name'] = Control_affli['first_name'] + ' ' + Control_affli['last_name']
MIMIC_affli['full_name'] = MIMIC_affli['first_name'] + ' ' + MIMIC_affli['last_name']


In [ ]:
MIMIC_affli[['aff_id', 'aff_name','pub_id', 'researcher_id','Author_Order' ]].head(10)

,aff_id,aff_name,pub_id,researcher_id,Author_Order
0,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.014116573320.23,First
1,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.016241001520.36,Middle
2,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.013553641416.27,Middle
3,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.014351222016.81,Middle
4,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.014227217615.79,First
5,grid.258164.c,Jinan University,pub.1154144039,ur.012755055514.47,Middle
6,grid.412601.0,First Affiliated Hospital of Jinan University,pub.1154144039,ur.012157101103.38,Last
7,grid.419653.c,National Institute of Technology Tiruchirappalli,pub.1154120926,ur.012301174053.60,First
8,grid.419653.c,National Institute of Technology Tiruchirappalli,pub.1154120926,ur.016340733111.17,Last
9,grid.43555.32,Beijing Institute of Technology,pub.1154113975,ur.011344015515.23,First


In [ ]:
MIMIC_affli.LMIC.value_counts()

No     13312
Yes      655
Name: LMIC, dtype: int64

In [ ]:
# Count the total number of rows in the dataframe
total_rows = len(MIMIC_affli)

# Count the number of rows where the author is from an LMIC
rows_with_lmic_author = len(MIMIC_affli[MIMIC_affli['LMIC'] == "Yes"])

# Calculate the proportion
proportion_with_lmic_author = (rows_with_lmic_author / total_rows) * 100

print(f"Total number of authors: {total_rows}")
print(f"Number of authors from an LMIC: {rows_with_lmic_author}")
print(f"Proportion of authors from an LMIC: {proportion_with_lmic_author:.2f}%")

# Count the total number of rows in the dataframe
total_rows = len(Control_affli)

# Count the number of rows where the author is from an LMIC
rows_with_lmic_author = len(Control_affli[Control_affli['LMIC'] == "Yes"])

# Calculate the proportion
proportion_with_lmic_author = (rows_with_lmic_author / total_rows) * 100

print(f"Total number of authors: {total_rows}")
print(f"Number of authors from an LMIC: {rows_with_lmic_author}")
print(f"Proportion of authors from an LMIC: {proportion_with_lmic_author:.2f}%")


Total number of authors: 15639
Number of authors from an LMIC: 655
Proportion of authors from an LMIC: 4.19%
Total number of authors: 25676
Number of authors from an LMIC: 756
Proportion of authors from an LMIC: 2.94%


In [ ]:
# Filter the rows where 'Author_Order' is 'First'
first_authors_df = MIMIC_affli[MIMIC_affli['Author_Order'] == 'First']
# Count the total number of first authors
total_first_authors = len(first_authors_df)
# Count the number of first authors from an LMIC
first_authors_from_lmic = len(first_authors_df[first_authors_df['LMIC'] == 'Yes'])
# Calculate the proportion of first authors from LMIC
proportion_first_authors_from_lmic = (first_authors_from_lmic / total_first_authors) * 100
print(f"MIMIC: Number of first authors from an LMIC: {first_authors_from_lmic}")
print(f"MIMIC: Total number of first authors: {total_first_authors }")
print(f"MIMIC:  Proportion of first authors from an LMIC: {proportion_first_authors_from_lmic:.2f}%")

# Filter the rows where 'Author_Order' is 'First'
first_authors_df = Control_affli[Control_affli['Author_Order'] == 'First']
# Count the total number of first authors
total_first_authors = len(first_authors_df)
# Count the number of first authors from an LMIC
first_authors_from_lmic = len(first_authors_df[first_authors_df['LMIC'] == 'Yes'])
# Calculate the proportion of first authors from LMIC
proportion_first_authors_from_lmic = (first_authors_from_lmic / total_first_authors) * 100
print(f"Control ： Number of first authors from an LMIC: {first_authors_from_lmic}")
print(f"Control: Total number of first authors: {total_first_authors }")
print(f"Control：  Proportion of first authors from an LMIC: {proportion_first_authors_from_lmic:.2f}%")


MIMIC: Number of first authors from an LMIC: 127
MIMIC: Total number of first authors: 2429
MIMIC:  Proportion of first authors from an LMIC: 5.23%
Control ： Number of first authors from an LMIC: 160
Control: Total number of first authors: 3945
Control：  Proportion of first authors from an LMIC: 4.06%


In [ ]:
# Filter the rows where 'Author_Order' is 'Last'
Last_authors_df = MIMIC_affli[MIMIC_affli['Author_Order'] == 'Last']
# Count the total number of Last authors
total_Last_authors = len(Last_authors_df)
# Count the number of Last authors from an LMIC
Last_authors_from_lmic = len(Last_authors_df[Last_authors_df['LMIC'] == 'Yes'])
# Calculate the proportion of Last authors from LMIC
proportion_Last_authors_from_lmic = (Last_authors_from_lmic / total_Last_authors) * 100
print(f"MIMIC ： Number of Last authors from an LMIC: {Last_authors_from_lmic}")
print(f"MIMIC: Total number of last authors: {total_Last_authors }")
print(f"MIMIC：  Proportion of Last authors from an LMIC: {proportion_Last_authors_from_lmic:.2f}%")

# Filter the rows where 'Author_Order' is 'Last'
Last_authors_df = Control_affli[Control_affli['Author_Order'] == 'Last']
# Count the total number of Last authors
total_Last_authors = len(Last_authors_df)
# Count the number of Last authors from an LMIC
Last_authors_from_lmic = len(Last_authors_df[Last_authors_df['LMIC'] == 'Yes'])
# Calculate the proportion of Last authors from LMIC
proportion_Last_authors_from_lmic = (Last_authors_from_lmic / total_Last_authors) * 100
print(f"Control ： Number of Last authors from an LMIC: {Last_authors_from_lmic}")
print(f"Control: Total number of first authors: {total_Last_authors }")
print(f"Control：  Proportion of Last authors from an LMIC: {proportion_Last_authors_from_lmic:.2f}%")


MIMIC ： Number of Last authors from an LMIC: 142
MIMIC: Total number of last authors: 3007
MIMIC：  Proportion of Last authors from an LMIC: 4.72%
Control ： Number of Last authors from an LMIC: 158
Control: Total number of first authors: 5193
Control：  Proportion of Last authors from an LMIC: 3.04%


In [ ]:
print(len(MIMIC_auth))
MIMIC_unique_authors = MIMIC_auth['researcher_id'].nunique()
print(f"There are {MIMIC_unique_authors} unique authors in the MIMIC_author dataframe.")
MIMIC_missing_researcher_ids = MIMIC_auth['researcher_id'].isna().sum()
print(f"There are {MIMIC_missing_researcher_ids} missing researcher_ids in the MIMIC_author dataframe.")
print(f"There are {len(MIMIC_auth)-MIMIC_missing_researcher_ids} records can be mapped ")

13838
There are 6606 unique authors in the MIMIC_author dataframe.
There are 3652 missing researcher_ids in the MIMIC_author dataframe.
There are 10186 records can be mapped 


In [ ]:
# MIMIC_author[MIMIC_author['researcher_id'].isna()]

In [ ]:
print(len(Control_author))
Control_unique_authors = Control_author['researcher_id'].nunique()
print(f"There are {Control_unique_authors} unique authors in the Control_author dataframe.")
Control_missing_researcher_ids = Control_author['researcher_id'].isna().sum()
print(f"There are {Control_missing_researcher_ids} missing researcher_ids in the Control_author dataframe.")
print(f"There are {len(Control_author)-Control_missing_researcher_ids} records can be mapped in Control  ")

21364
There are 15186 unique authors in the Control_author dataframe.
There are 3015 missing researcher_ids in the Control_author dataframe.
There are 18349 records can be mapped in Control  


In [ ]:
print(13879+20539)
print(6766+14821)
print(10361+17742)
print(28103/34418)

34418
21587
28103
0.8165204253588239


In [ ]:
print("MIMIC affli author id unique",MIMIC_affli.researcher_id.nunique()) # ??
print("Control affli author id unique",Control_affli.researcher_id.nunique())


MIMIC affli author id unique 5999
Control affli author id unique 14270


In [ ]:
set(MIMIC_affli.pub_id).difference(set(MIMIC_pubs.id))



set()

In [ ]:
# Get unique pub_id in MIMIC_affli
unique_pub_id_in_MIMIC_affli = set(MIMIC_affli['pub_id'].dropna().unique())

# Get unique pub_id in MIMIC_author
unique_pub_id_in_MIMIC_pubs = set(MIMIC_pubs['id'].dropna().unique())

# Get intersection of unique pub_id in both MIMIC_affli and MIMIC_author
MIMIC_intersection_ids = unique_pub_id_in_MIMIC_affli.intersection(unique_pub_id_in_MIMIC_pubs)
print(len(MIMIC_intersection_ids))

1672


In [ ]:
# Get unique researcher_id in MIMIC_affli
unique_researcher_id_in_MIMIC_affli = set(MIMIC_affli['researcher_id'].dropna().unique())

# Get unique researcher_id in MIMIC_author
unique_researcher_id_in_MIMIC_author = set(MIMIC_author['researcher_id'].dropna().unique())

# Get intersection of unique researcher_id in both MIMIC_affli and MIMIC_author
MIMIC_intersection_ids = unique_researcher_id_in_MIMIC_affli.intersection(unique_researcher_id_in_MIMIC_author)

# Print number of intersection researcher_id
print(f"The number of unique researcher_id present in both MIMIC_affli and MIMIC_author is: {len(MIMIC_intersection_ids)}")
# Get unique researcher_id in Control_affli
unique_researcher_id_in_Control_affli = set(Control_affli['researcher_id'].dropna().unique())

# Get unique researcher_id in Control_author
unique_researcher_id_in_Control_author = set(Control_author['researcher_id'].dropna().unique())

# Get intersection of unique researcher_id in both Control_affli and Control_author
Control_intersection_ids = unique_researcher_id_in_Control_affli.intersection(unique_researcher_id_in_Control_author)

# Print number of intersection researcher_id
print(f"The number of unique researcher_id present in both Control_affli and Control_author is: {len(Control_intersection_ids)}")


The number of unique researcher_id present in both MIMIC_affli and MIMIC_author is: 5999
The number of unique researcher_id present in both Control_affli and Control_author is: 14270


In [ ]:
len(MIMIC_intersection_ids.intersection(Control_intersection_ids))

562

In [ ]:
# Assuming the MIMIC_pubs dataframe is named 'MIMIC_pubs'
# Group the dataframe by 'journal.title' and count the number of publications for each journal
journal_counts = MIMIC_pubs.groupby('journal.title').size()

# Sort the journals by the number of publications in descending order
sorted_journal_counts = journal_counts.sort_values(ascending=False)

# Get the top 10 journals
top_10_journals = sorted_journal_counts.head(10)

print("MIMIC Top 10 journals by the number of publications:")
print(top_10_journals)

# import pandas as pd
# import matplotlib.pyplot as plt
# # Group the dataframe by 'journal.title' and count the number of publications for each journal
# journal_counts = MIMIC_pubs.groupby('journal.title').size()

# # Sort the journals by the number of publications in descending order
# sorted_journal_counts = journal_counts.sort_values(ascending=False)

# # Get the top 10 journals
# top_10_journals = sorted_journal_counts.head(10)

# # Create a bar plot of the top 10 journals
# fig, ax = plt.subplots(figsize=(10, 6))
# top_10_journals.plot(kind='bar', ax=ax)
# ax.set_ylabel('Number of Publications')
# ax.set_xlabel('Journal Title')
# ax.set_title('Top 10 Journals by Number of Publications')
# plt.xticks(rotation=45, ha='right')

# # Show the plot
# plt.show()

MIMIC Top 10 journals by the number of publications:
journal.title
arXiv                                          475
Research Square                                148
medRxiv                                         53
Frontiers in Medicine                           42
Scientific Reports                              31
Critical Care Medicine                          29
SSRN Electronic Journal                         25
Frontiers in Cardiovascular Medicine            24
PLOS ONE                                        24
BMC Medical Informatics and Decision Making     22
dtype: int64


In [ ]:

# Group the dataframe by 'journal.title' and count the number of publications for each journal
journal_counts = Control_pubs.groupby('journal.title').size()

# Sort the journals by the number of publications in descending order
sorted_journal_counts = journal_counts.sort_values(ascending=False)

# Get the top 10 journals
top_10_journals = sorted_journal_counts.head(10)

print("Control Top 10 journals by the number of publications:")
print(top_10_journals)

# import pandas as pd
# import matplotlib.pyplot as plt
# # Group the dataframe by 'journal.title' and count the number of publications for each journal
# journal_counts = Control_pubs.groupby('journal.title').size()

# # Sort the journals by the number of publications in descending order
# sorted_journal_counts = journal_counts.sort_values(ascending=False)

# # Get the top 10 journals
# top_10_journals = sorted_journal_counts.head(10)

# # Create a bar plot of the top 10 journals
# fig, ax = plt.subplots(figsize=(10, 6))
# top_10_journals.plot(kind='bar', ax=ax)
# ax.set_ylabel('Number of Publications')
# ax.set_xlabel('Journal Title')
# ax.set_title('Top 10 Journals by Number of Publications')
# plt.xticks(rotation=45, ha='right')

# # Show the plot
# plt.show()

Control Top 10 journals by the number of publications:
journal.title
Annual International Conference of the IEEE Engineering in Medicine and Biology Society (EMBC)    99
PLOS ONE                                                                                          71
Scientific Reports                                                                                68
Critical Care Medicine                                                                            68
Journal of Clinical Monitoring and Computing                                                      44
Computers in Biology and Medicine                                                                 35
Computer Methods and Programs in Biomedicine                                                      32
Sensors                                                                                           32
Critical Care                                                                                     32
Artificial Intelligenc

In [ ]:
MIMIC_author['researcher_id'].value_counts()

ur.01144416340.69     60
ur.012157101103.38    40
ur.0616057325.38      27
ur.011417072171.82    24
ur.011617401167.17    24
                      ..
ur.011352461233.38     1
ur.012751232000.92     1
ur.013546612400.88     1
ur.01324061732.13      1
ur.012014630262.69     1
Name: researcher_id, Length: 6766, dtype: int64

In [ ]:

# Create a boolean column indicating if a paper has a missing affiliation for any author
MIMIC_affli['missing_affiliation'] = MIMIC_affli['aff_country'].isna() | MIMIC_affli['aff_country_code'].isna()

# Group by publication and aggregate the 'missing_affiliation' column with 'any' function
grouped_affiliations = MIMIC_affli.groupby('pub_id')['missing_affiliation'].agg('any').reset_index(name='any_missing_affiliation')

# Filter the papers with non-missing affiliations
papers_with_non_missing_affiliations = grouped_affiliations[grouped_affiliations['any_missing_affiliation'] == False]

# Calculate the percentage of papers with at least one author with a missing affiliation
num_papers = len(grouped_affiliations)
num_papers_with_missing_affiliation = grouped_affiliations['any_missing_affiliation'].sum()
percentage_papers_with_missing_affiliation = (num_papers_with_missing_affiliation / num_papers) * 100

print(f"Number of papers with non-missing affiliations: {len(papers_with_non_missing_affiliations)}")
print(f"{percentage_papers_with_missing_affiliation:.2f}% of papers had at least one author with a missing affiliation.")


Number of papers with non-missing affiliations: 1487
11.06% of papers had at least one author with a missing affiliation.


In [ ]:
MIMIC_affli.groupby('pub_id').agg('any')

,Unnamed: 0.1,Unnamed: 0,aff_city,aff_city_id,aff_country,aff_country_code,aff_id,aff_name,aff_raw_affiliation,aff_state,...,first_name,last_name,year,gender,gender_ints,income_class,income_class_num,missing_affiliation,LMIC,full_name
pub_id,,,,,,,,,,,,,,,,,,,,,
pub.1001792778,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,False,True,True
pub.1002353037,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,True,True,False,True,True
pub.1003040860,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
pub.1006639384,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,False,True,True
pub.1007449969,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pub.1155896423,True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,False,True,True
pub.1155904804,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,False,True,True
pub.1156298472,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
MIMIC_auth.groupby('pub_id').agg('any')

,Unnamed: 0.1,Unnamed: 0,affiliations,corresponding,current_organization_id,first_name,last_name,orcid,raw_affiliation,researcher_id,year,gender,gender_ints,Country,CountryIncomeClass,Affliation_id
pub_id,,,,,,,,,,,,,,,,
pub.1001792778,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
pub.1002353037,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True
pub.1003040860,True,True,True,True,False,True,True,False,True,True,True,True,True,True,True,True
pub.1005815647,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True
pub.1006639384,True,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pub.1155035716,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
pub.1155035818,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
pub.1155035834,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [ ]:
MIMIC_pubs.groupby('id').agg('any')

,Unnamed: 0,title,abstract,authors,authors_count,category_for,category_hra,category_hrcs_hc,category_hrcs_rac,category_rcdc,...,num_na,pct_female_real,female_first,female_last,pct_female_pessimistic,pct_female_optimistic,female_first_pessimistic,female_first_optimistic,female_last_pessimistic,female_last_optimistic
id,,,,,,,,,,,,,,,,,,,,,
pub.1001792778,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
pub.1002353037,True,True,True,True,True,True,True,True,True,True,...,True,False,False,False,False,True,False,True,False,True
pub.1003040860,True,True,True,True,True,True,True,False,False,False,...,False,True,True,True,True,True,True,True,True,True
pub.1005815647,True,True,False,True,True,True,False,False,False,False,...,True,True,False,False,True,True,False,True,False,False
pub.1006639384,True,True,True,True,True,True,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pub.1155896423,True,True,True,True,True,True,False,False,True,True,...,True,True,False,True,True,True,False,True,True,True
pub.1155904804,True,True,True,True,True,True,True,False,True,True,...,True,True,False,False,True,True,False,True,False,False
pub.1156298472,True,True,True,True,True,True,False,False,False,False,...,True,True,False,True,True,True,False,False,True,True


### based on pubs

In [ ]:
MIMIC_affli.pub_id.value_counts()

MIMIC_author_counts = MIMIC_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with only one author
MIMIC_single_author_papers = MIMIC_author_counts[MIMIC_author_counts == 1].count()

print(f"MIMIC Number of papers with only one author: {MIMIC_single_author_papers}")

MIMIC Number of papers with only one author: 104


In [ ]:
Control_affli.pub_id.value_counts()


# Assuming 'df' is your DataFrame with 'pub_id' and 'researcher_id' columns
# Group by 'pub_id' and count the number of unique authors for each publication
Control_author_counts = Control_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with only one author
Control_single_author_papers = Control_author_counts[Control_author_counts == 1].count()

print(f"Control Number of papers with only one author: {Control_single_author_papers}")

Control Number of papers with only one author: 178


In [ ]:
CountryClass['Income group'].value_counts()

High income            80
Lower middle income    55
Upper middle income    55
Low income             27
Name: Income group, dtype: int64

In [ ]:
Control_affli.income_class.value_counts()

High income            19798
Upper middle income     4278
Lower middle income      742
Low income                14
Name: income_class, dtype: int64

## Any LMIC count

In [ ]:
# Convert 'Yes' and 'No' in 'LMIC' column to 1 and 0
# MIMIC_affli['LMIC'] = MIMIC_affli['LMIC'].map({'Yes': 1, 'No': 0}) # Note this line should be only use one time

# Group the dataframe by 'pub_id' and count the number of authors for each publication
author_counts = MIMIC_affli.groupby('pub_id').size()

# Get the 'pub_id' of papers with two or more authors
papers_with_two_or_more_authors = author_counts[author_counts >= 2].index

# Filter the dataframe to keep only papers with two or more authors
MIMIC_affli_two_or_more_authors = MIMIC_affli[MIMIC_affli['pub_id'].isin(papers_with_two_or_more_authors)]

# Count the number of LMIC authors for each paper
lmic_counts = MIMIC_affli_two_or_more_authors.groupby('pub_id')['LMIC'].sum()

# Determine the papers with multiple LMIC authors
papers_with_multiple_lmic_authors = lmic_counts[lmic_counts >= 2].index

# Calculate the percentage
percentage_multiple_lmic_authors = (len(papers_with_multiple_lmic_authors) / len(papers_with_two_or_more_authors)) * 100

print(f"MIMIC  Among articles with two authors or more, {percentage_multiple_lmic_authors:.2f}% papers had multiple LMIC authors.")

# Repeat the same process for the Control_affli DataFrame
# Convert 'Yes' and 'No' in 'LMIC' column to 1 and 0
# Control_affli['LMIC'] = Control_affli['LMIC'].map({'Yes': 1, 'No': 0})

# Group the dataframe by 'pub_id' and count the number of authors for each publication
author_counts = Control_affli.groupby('pub_id').size()

# Get the 'pub_id' of papers with two or more authors
papers_with_two_or_more_authors = author_counts[author_counts >= 2].index

# Filter the dataframe to keep only papers with two or more authors
Control_affli_two_or_more_authors = Control_affli[Control_affli['pub_id'].isin(papers_with_two_or_more_authors)]

# Count the number of LMIC authors for each paper
lmic_counts = Control_affli_two_or_more_authors.groupby('pub_id')['LMIC'].sum()

# Determine the papers with multiple LMIC authors
papers_with_multiple_lmic_authors = lmic_counts[lmic_counts >= 2].index

# Calculate the percentage
percentage_multiple_lmic_authors = (len(papers_with_multiple_lmic_authors) / len(papers_with_two_or_more_authors)) * 100

print(f"Control  Among articles with two authors or more, {percentage_multiple_lmic_authors:.2f}% papers had multiple LMIC authors.")

# Repeat the same process for the Control_affli DataFrame


MIMIC  Among articles with two authors or more, 8.32% papers had multiple LMIC authors.
Control  Among articles with two authors or more, 5.51% papers had multiple LMIC authors.


In [ ]:
print("Control")
Control_lmic_count = Control_affli[Control_affli['LMIC'] == 1].groupby('pub_id').agg({'LMIC': 'count'}).reset_index()
print("LMIC>0",len(Control_lmic_count[Control_lmic_count.LMIC>0]))
print("MIMIC")
MIMIC_lmic_count = MIMIC_affli[MIMIC_affli['LMIC'] == 1].groupby('pub_id').agg({'LMIC': 'count'}).reset_index()
print("LMIC>0",len(MIMIC_lmic_count[MIMIC_lmic_count.LMIC>0]))

Control
LMIC>0 182
MIMIC
LMIC>0 156


In [ ]:
len(MIMIC_lmic_count)

156

In [ ]:
IncomeByPub = MIMIC_affli[['pub_id','income_class','income_class_num']].groupby(['pub_id']).agg([ 'count','mean']).reset_index()

<ipython-input-165-3edde4cf4347>:1: FutureWarning: ['income_class'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  IncomeByPub = MIMIC_affli[['pub_id','income_class','income_class_num']].groupby(['pub_id']).agg([ 'count','mean']).reset_index()


In [ ]:
MIMIC_affli[['pub_id','income_class_num']].groupby(['pub_id']).mean().reset_index()

,pub_id,income_class_num
0,pub.1001792778,4.0
1,pub.1002353037,4.0
2,pub.1003040860,4.0
3,pub.1006639384,4.0
4,pub.1007449969,3.0
...,...,...
1667,pub.1155896423,2.0
1668,pub.1155904804,3.0
1669,pub.1156298472,2.0
1670,pub.1156298670,2.8


In [ ]:
# Determine unique 'pub_id's with at least one LMIC author
lmic_pubs = MIMIC_affli[MIMIC_affli['LMIC'] == 1]['pub_id'].nunique()

# Calculate the total number of unique 'pub_id's
total_pubs = MIMIC_affli['pub_id'].nunique()

# Calculate the percentage
percentage_lmic = (lmic_pubs / total_pubs) * 100

print(f"Overall, {percentage_lmic:.2f}% of articles in the considered set had at least one LMIC author.")

# Repeat the same for Control dataset

lmic_pubs_control = Control_affli[Control_affli['LMIC'] == 1]['pub_id'].nunique()
total_pubs_control = Control_affli['pub_id'].nunique()
percentage_lmic_control = (lmic_pubs_control / total_pubs_control) * 100

print(f"Overall, {percentage_lmic_control:.2f}% of Control articles in the considered set had at least one LMIC author.")

# Concatenate the two DataFrames
combined_affli = pd.concat([MIMIC_affli, Control_affli])

# Determine unique 'pub_id's with at least one LMIC author
lmic_pubs_combined = combined_affli[combined_affli['LMIC'] == 1]['pub_id'].nunique()

# Calculate the total number of unique 'pub_id's
total_pubs_combined = combined_affli['pub_id'].nunique()

# Calculate the percentage
percentage_lmic_combined = (lmic_pubs_combined / total_pubs_combined) * 100

print(f"Overall, {percentage_lmic_combined:.2f}% of articles in the combined set had at least one LMIC author.")


Overall, 9.33% of articles in the considered set had at least one LMIC author.
Overall, 6.33% of Control articles in the considered set had at least one LMIC author.
Overall, 7.43% of articles in the combined set had at least one LMIC author.


In [ ]:
print("MIMIC/eICU")
papers_with_two_or_more_authors = MIMIC_affli.groupby('pub_id').filter(lambda x: len(x) >= 2)

# Group papers by publication id and count the number of LMIC authors per paper
lmic_counts = papers_with_two_or_more_authors[papers_with_two_or_more_authors['LMIC'] == 1].groupby('pub_id')['LMIC'].count()
print("total lmic", len(lmic_counts))
# Calculate the percentage of papers with multiple LMIC authors
papers_with_multiple_lmic_authors = lmic_counts[lmic_counts >= 2].index
percentage_multiple_lmic_authors = (len(papers_with_multiple_lmic_authors) / len(papers_with_two_or_more_authors['pub_id'].unique())) * 100

# Determine papers where the first author is affiliated with a LMIC country
first_authors = papers_with_two_or_more_authors.groupby('pub_id').first()

papers_with_first_author_lmic = first_authors[first_authors['LMIC'] == 1].index
print("first_authors",len(papers_with_first_author_lmic))
# Calculate the percentage of papers where the first author is affiliated with a LMIC country
percentage_first_author_lmic = (len(papers_with_first_author_lmic) / len(papers_with_two_or_more_authors['pub_id'].unique())) * 100

print(f"Among articles with two authors or more, {percentage_multiple_lmic_authors:.2f}% had multiple LMIC authors.")
print(f"Additionally, the first author was affiliated with a LMIC country in {percentage_first_author_lmic:.2f}% of papers with two or more authors.")


MIMIC/eICU
total lmic 154
first_authors 118
Among articles with two authors or more, 8.32% had multiple LMIC authors.
Additionally, the first author was affiliated with a LMIC country in 7.16% of papers with two or more authors.


In [ ]:
print("Control")
papers_with_two_or_more_authors = Control_affli.groupby('pub_id').filter(lambda x: len(x) >= 2)

# Group papers by publication id and count the number of LMIC authors per paper
lmic_counts = papers_with_two_or_more_authors[papers_with_two_or_more_authors['LMIC'] == 1].groupby('pub_id')['LMIC'].count()
print("total lmic", len(lmic_counts))
# Calculate the percentage of papers with multiple LMIC authors
papers_with_multiple_lmic_authors = lmic_counts[lmic_counts >= 2].index
percentage_multiple_lmic_authors = (len(papers_with_multiple_lmic_authors) / len(papers_with_two_or_more_authors['pub_id'].unique())) * 100

# Determine papers where the first author is affiliated with a LMIC country
first_authors = papers_with_two_or_more_authors.groupby('pub_id').first()

papers_with_first_author_lmic = first_authors[first_authors['LMIC'] == 1].index
print("first_authors",len(papers_with_first_author_lmic))
# Calculate the percentage of papers where the first author is affiliated with a LMIC country
percentage_first_author_lmic = (len(papers_with_first_author_lmic) / len(papers_with_two_or_more_authors['pub_id'].unique())) * 100

print(f"Among articles with two authors or more, {percentage_multiple_lmic_authors:.2f}% had multiple LMIC authors.")
print(f"Additionally, the first author was affiliated with a LMIC country in {percentage_first_author_lmic:.2f}% of papers with two or more authors.")


Control
total lmic 175
first_authors 131
Among articles with two authors or more, 5.51% had multiple LMIC authors.
Additionally, the first author was affiliated with a LMIC country in 4.78% of papers with two or more authors.


### Statistic Analysis

In [ ]:
MIMIC_first_authors = MIMIC_affli.groupby('pub_id').first().reset_index()
MIMIC_last_authors = MIMIC_affli.groupby('pub_id').last().reset_index()
MIMIC_first_author_lmic_count = MIMIC_first_authors[MIMIC_first_authors['LMIC'] == 1].shape[0]
MIMIC_last_author_lmic_count = MIMIC_last_authors[MIMIC_last_authors['LMIC'] == 1].shape[0]

MIMIC_lmic_authors = MIMIC_affli[MIMIC_affli['LMIC'] == 1]

# Drop duplicate 'pub_id' values to count unique publications
MIMIC_unique_lmic_papers = MIMIC_lmic_authors.drop_duplicates(subset='pub_id')

# Count the number of unique publications with at least one LMIC author
MIMIC_num_lmic_papers = len(MIMIC_unique_lmic_papers)

print(f"MIMIC Number of publications with any author from LMIC: {MIMIC_num_lmic_papers}")
print(f"MIMIC Number of first authors from LMIC: {MIMIC_first_author_lmic_count}")
print(f"MIMIC Number of last authors from LMIC: {MIMIC_last_author_lmic_count}")
print(f"MIMIC Number of papers with only one author: {MIMIC_single_author_papers}")

MIMIC Number of publications with any author from LMIC: 156
MIMIC Number of first authors from LMIC: 120
MIMIC Number of last authors from LMIC: 120
MIMIC Number of papers with only one author: 104


In [ ]:
Control_first_authors = Control_affli.groupby('pub_id').first().reset_index()
Control_last_authors = Control_affli.groupby('pub_id').last().reset_index()
Control_first_author_lmic_count = Control_first_authors[Control_first_authors['LMIC'] == 1].shape[0]
Control_last_author_lmic_count = Control_last_authors[Control_last_authors['LMIC'] == 1].shape[0]

Control_lmic_authors = Control_affli[Control_affli['LMIC'] == 1]

# Drop duplicate 'pub_id' values to count unique publications
Control_unique_lmic_papers = Control_lmic_authors.drop_duplicates(subset='pub_id')

# Count the number of unique publications with at least one LMIC author
Control_num_lmic_papers = len(Control_unique_lmic_papers)

print(f"Control Number of publications with any author from LMIC: {Control_num_lmic_papers}")
print(f"Control Number of first authors from LMIC: {Control_first_author_lmic_count}")
print(f"Control Number of last authors from LMIC: {Control_last_author_lmic_count}")
print(f"Control Number of papers with only one author: {Control_single_author_papers}")

Control Number of publications with any author from LMIC: 182
Control Number of first authors from LMIC: 138
Control Number of last authors from LMIC: 127
Control Number of papers with only one author: 178


In [ ]:
#Overall
print(156/1672)
print(120/1672)
print(120/1672)
print(182/2877)
print(138/2877)
print(127/2877)

0.09330143540669857
0.07177033492822966
0.07177033492822966
0.06326034063260341
0.047966631908237745
0.044143204727146335


In [ ]:
Control_num_lmic_papers

182

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
# Number of successes (LMIC authors) in each group
counts = np.array([156, 182])

# Total number of observations in each group
nobs = np.array([1672, 2877])

# Perform the Z-test with the alternative hypothesis stating that the proportion in the MIMIC/eICU group is greater than the control group
z_stat, p_value = proportions_ztest(counts, nobs, alternative='larger')

print(f"Z-stat: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

Z-stat: 3.7249
P-value: 0.0001


In [ ]:
# Filter the DataFrame to get rows with LMIC authors
lmic_authors = MIMIC_affli[MIMIC_affli['LMIC'] == 1]

# Group by 'pub_id' and count the number of LMIC authors for each publication
lmic_author_counts = lmic_authors.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more LMIC authors
multiple_lmic_papers = lmic_author_counts[lmic_author_counts >= 2].count()

# Group by 'pub_id' and count the total number of authors for each publication
total_author_counts = MIMIC_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more authors
two_or_more_authors_papers = total_author_counts[total_author_counts >= 2].count()

# Calculate the percentage of papers with multiple LMIC authors among those with two or more authors
percentage_multiple_lmic = (multiple_lmic_papers / two_or_more_authors_papers) * 100

print(f"MIMIC Percentage of papers with multiple LMIC authors among those with two or more authors: {percentage_multiple_lmic:.2f}%")


MIMIC Percentage of papers with multiple LMIC authors among those with two or more authors: 7.94%


In [ ]:
# Filter the DataFrame to get rows with LMIC authors
lmic_authors = Control_affli[Control_affli['LMIC'] == 1]

# Group by 'pub_id' and count the number of LMIC authors for each publication
lmic_author_counts = lmic_authors.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more LMIC authors
multiple_lmic_papers = lmic_author_counts[lmic_author_counts >= 2].count()

# Group by 'pub_id' and count the total number of authors for each publication
total_author_counts = Control_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more authors
two_or_more_authors_papers = total_author_counts[total_author_counts >= 2].count()

# Calculate the percentage of papers with multiple LMIC authors among those with two or more authors
percentage_multiple_lmic = (multiple_lmic_papers / two_or_more_authors_papers) * 100

print(f"Control Percentage of papers with multiple LMIC authors among those with two or more authors: {percentage_multiple_lmic:.2f}%")


Control Percentage of papers with multiple LMIC authors among those with two or more authors: 5.42%


In [ ]:
import pandas as pd

# Group by 'pub_id' and count the total number of authors for each publication
total_author_counts = MIMIC_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more authors
two_or_more_authors_papers = total_author_counts[total_author_counts >= 2].index

# Get a new DataFrame with only papers with two or more authors
two_or_more_authors_df = MIMIC_affli[MIMIC_affli['pub_id'].isin(two_or_more_authors_papers)]

# Sort the DataFrame by 'pub_id' and reset the index
sorted_df = two_or_more_authors_df.sort_values(by='pub_id').reset_index(drop=True)

# Get the first and  author for each publication
first_authors = sorted_df.groupby('pub_id').first()
last_authors = sorted_df.groupby('pub_id').last()
# Filter the DataFrame to get rows with LMIC first authors
lmic_first_authors = first_authors[first_authors['LMIC'] == 1]
lmic_last_authors = last_authors[last_authors['LMIC']==1]
# Count the number of papers with the first author from LMIC
num_MIMIC_lmic_first_authors = len(lmic_first_authors)
num_MIMIC_lmic_last_authors = len(lmic_last_authors)
# Count the total number of papers with two or more authors
total_papers_two_or_more_authors = len(two_or_more_authors_papers)

# Calculate the percentage of papers with the first author from LMIC among those with two or more authors
percentage_lmic_first_authors = (num_MIMIC_lmic_first_authors / total_papers_two_or_more_authors) * 100
percentage_lmic_last_authors = (num_MIMIC_lmic_first_authors / total_papers_two_or_more_authors) * 100

print(f"MIMIC Percentage of papers with two or more authors having the first author from LMIC: {percentage_lmic_first_authors:.2f}%")
print(f"MIMIC Percentage of papers with two or more authors having the last author from LMIC: {percentage_lmic_last_authors:.2f}%")

MIMIC Percentage of papers with two or more authors having the first author from LMIC: 7.14%
MIMIC Percentage of papers with two or more authors having the last author from LMIC: 7.14%


In [ ]:

# Group by 'pub_id' and count the total number of authors for each publication
total_author_counts = Control_affli.groupby('pub_id')['researcher_id'].nunique()

# Filter the results to get the number of papers with two or more authors
two_or_more_authors_papers = total_author_counts[total_author_counts >= 2].index

# Get a new DataFrame with only papers with two or more authors
two_or_more_authors_df = Control_affli[Control_affli['pub_id'].isin(two_or_more_authors_papers)]

# Sort the DataFrame by 'pub_id' and reset the index
sorted_df = two_or_more_authors_df.sort_values(by='pub_id').reset_index(drop=True)

# Get the first and  author for each publication
first_authors = sorted_df.groupby('pub_id').first()
last_authors = sorted_df.groupby('pub_id').last()
# Filter the DataFrame to get rows with LMIC first authors
Control_lmic_first_authors = first_authors[first_authors['LMIC'] == 1]
Control_lmic_last_authors = last_authors[last_authors['LMIC']==1]
# Count the number of papers with the first author from LMIC
num_Control_lmic_first_authors = len(Control_lmic_first_authors)
num_Control_lmic_last_authors = len(Control_lmic_last_authors)
# Count the total number of papers with two or more authors
total_papers_two_or_more_authors = len(two_or_more_authors_papers)

# Calculate the percentage of papers with the first author from LMIC among those with two or more authors
percentage_lmic_first_authors = (num_Control_lmic_first_authors / total_papers_two_or_more_authors) * 100
percentage_lmic_last_authors = (num_Control_lmic_last_authors / total_papers_two_or_more_authors) * 100

print(f"Control Percentage of papers with two or more authors having the first author from LMIC: {percentage_lmic_first_authors:.3f}%")
print(f"Control Percentage of papers with two or more authors having the last author from LMIC: {percentage_lmic_last_authors:.3f}%")

Control Percentage of papers with two or more authors having the first author from LMIC: 4.593%
Control Percentage of papers with two or more authors having the last author from LMIC: 4.744%


In [ ]:
# Concatenate the two DataFrames
combined_affli = pd.concat([MIMIC_affli, Control_affli])

# Determine unique 'pub_id's with at least one LMIC author
lmic_pubs_combined = combined_affli[combined_affli['LMIC'] == 1]['pub_id'].nunique()

# Calculate the total number of unique 'pub_id's
total_pubs_combined = combined_affli['pub_id'].nunique()

# Calculate the percentage
percentage_lmic_combined = (lmic_pubs_combined / total_pubs_combined) * 100

print(f"Overall, {percentage_lmic_combined:.2f}% of articles in the combined set had at least one LMIC author.")
# Gender intersection

# Filter the dataframe to keep only those papers with non-missing gender and LMIC status information
combined_affli_filtered = combined_affli.dropna(subset=['gender', 'LMIC'])

# Group the dataframe by 'pub_id' and check if there is at least one woman author and at least one LMIC-based author for each paper
papers_with_woman_and_lmic_authors = combined_affli_filtered.groupby('pub_id').apply(lambda x: (x['gender'].str.lower() == 'female').any() and (x['LMIC']==1).any())

# Calculate the number of papers with at least one woman and one LMIC-based researcher among the authors
papers_count = papers_with_woman_and_lmic_authors.sum()

# Calculate the total number of papers with non-missing gender and LMIC status information
total_papers = len(papers_with_woman_and_lmic_authors)

# Calculate the percentage
percentage_woman_and_lmic_authors = (papers_count / total_papers) * 100

print(f"combined Among the {total_papers} papers with non-missing gender and LMIC status information, {percentage_woman_and_lmic_authors:.2f}% had at least one woman and one LMIC-based researcher among the authors.")

Overall, 7.43% of articles in the combined set had at least one LMIC author.
combined Among the 4229 papers with non-missing gender and LMIC status information, 5.04% had at least one woman and one LMIC-based researcher among the authors.


In [ ]:
# Combine MIMIC and Control datasets
combined_affli = pd.concat([MIMIC_affli, Control_affli])

# Group by 'pub_id' and count the total number of authors for each publication
total_author_counts = combined_affli.groupby('pub_id').size()

# Filter the results to get the number of papers with two or more authors
two_or_more_authors_papers = total_author_counts[total_author_counts >= 2].index

# Get a new DataFrame with only papers with two or more authors
two_or_more_authors_df = combined_affli[combined_affli['pub_id'].isin(two_or_more_authors_papers)]

# Get the first and last author for each publication
first_authors = two_or_more_authors_df.groupby('pub_id').first()
last_authors = two_or_more_authors_df.groupby('pub_id').last()

# Filter the DataFrame to get rows with LMIC first authors and last authors
lmic_first_authors = first_authors[first_authors['LMIC'] == 1]
lmic_last_authors = last_authors[last_authors['LMIC'] == 1]

# Count the number of papers with the first author and the last author from LMIC
num_lmic_first_authors = lmic_first_authors['LMIC'].count()
num_lmic_last_authors = lmic_last_authors['LMIC'].count()

# Calculate the percentage of papers with the first author and the last author from LMIC among those with two or more authors
percentage_lmic_first_authors = (num_lmic_first_authors / total_author_counts.count()) * 100
percentage_lmic_last_authors = (num_lmic_last_authors / total_author_counts.count()) * 100

print(f"Combined Percentage of papers with two or more authors having the first author from LMIC: {percentage_lmic_first_authors:.2f}%")
print(f"Combined Percentage of papers with two or more authors having the last author from LMIC: {percentage_lmic_last_authors:.2f}%")


Combined Percentage of papers with two or more authors having the first author from LMIC: 5.47%
Combined Percentage of papers with two or more authors having the last author from LMIC: 5.23%


In [ ]:


# Filter the dataframe to keep only those papers with non-missing gender and LMIC status information
MIMIC_affli_filtered = MIMIC_affli.dropna(subset=['gender', 'LMIC'])

# Group the dataframe by 'pub_id' and check if there is at least one woman author and at least one LMIC-based author for each paper
papers_with_woman_and_lmic_authors = MIMIC_affli_filtered.groupby('pub_id').apply(lambda x: (x['gender'].str.lower() == 'female').any() and (x['LMIC']==1).any())

# Calculate the number of papers with at least one woman and one LMIC-based researcher among the authors
papers_count = papers_with_woman_and_lmic_authors.sum()

# Calculate the total number of papers with non-missing gender and LMIC status information
total_papers = len(papers_with_woman_and_lmic_authors)

# Calculate the percentage
percentage_woman_and_lmic_authors = (papers_count / total_papers) * 100

print(f"MIMIC Among the {total_papers} papers with non-missing gender and LMIC status information, {percentage_woman_and_lmic_authors:.2f}% had at least one woman and one LMIC-based researcher among the authors.")
# Filter the dataframe to keep only those papers with non-missing gender and LMIC status information
Control_affli_filtered = Control_affli.dropna(subset=['gender', 'LMIC'])

# Group the dataframe by 'pub_id' and check if there is at least one woman author and at least one LMIC-based author for each paper
papers_with_woman_and_lmic_authors = Control_affli_filtered.groupby('pub_id').apply(lambda x: (x['gender'].str.lower() == 'female').any() and (x['LMIC']==1).any())

# Calculate the number of papers with at least one woman and one LMIC-based researcher among the authors
papers_count = papers_with_woman_and_lmic_authors.sum()

# Calculate the total number of papers with non-missing gender and LMIC status information
total_papers = len(papers_with_woman_and_lmic_authors)

# Calculate the percentage
percentage_woman_and_lmic_authors = (papers_count / total_papers) * 100

print(f"Control Among the {total_papers} papers with non-missing gender and LMIC status information, {percentage_woman_and_lmic_authors:.2f}% had at least one woman and one LMIC-based researcher among the authors.")

MIMIC Among the 1603 papers with non-missing gender and LMIC status information, 6.11% had at least one woman and one LMIC-based researcher among the authors.
Control Among the 2626 papers with non-missing gender and LMIC status information, 4.38% had at least one woman and one LMIC-based researcher among the authors.


In [ ]:

# Proportions provided
mimic_proportion = 6.11 / 100
control_proportion = 4.38 / 100

# Total number of papers with non-missing gender and LMIC status information
mimic_total_papers = 1672
control_total_papers = 2877

# Calculate the number of papers with at least one woman and one LMIC-based researcher among the authors
mimic_papers_count = int(mimic_total_papers * mimic_proportion)
control_papers_count = int(control_total_papers * control_proportion)

# Perform the two-proportion Z-test
z_stat, p_value = proportions_ztest([mimic_papers_count, control_papers_count],
                                    [mimic_total_papers, control_total_papers],
                                    alternative='larger')

print(f"one-sided Two-proportion Z-test: p-value = {p_value:.4f}")

one-sided Two-proportion Z-test: p-value = 0.0052


In [ ]:
# Combine both DataFrames: MIMIC/eICU and control datasets
combined_affli = pd.concat([MIMIC_affli, Control_affli])



# Filter the dataframe to keep only those papers with non-missing gender and LMIC status information
Combined_affli_filtered = combined_affli.dropna(subset=['gender', 'LMIC'])

# Group the dataframe by 'pub_id' and check if there is at least one woman author and at least one LMIC-based author for each paper
papers_with_woman_and_lmic_authors = Combined_affli_filtered.groupby('pub_id').apply(lambda x: (x['gender'].str.lower() == 'female').any() and (x['LMIC']=='Yes').any())

# Calculate the number of papers with at least one woman and one LMIC-based researcher among the authors
papers_count = papers_with_woman_and_lmic_authors.sum()

# Calculate the total number of papers with non-missing gender and LMIC status information
total_papers = len(papers_with_woman_and_lmic_authors)

# Calculate the percentage
percentage_woman_and_lmic_authors = (papers_count / total_papers) * 100

print(f"Combined Among the {total_papers} papers with non-missing gender and LMIC status information, {percentage_woman_and_lmic_authors:.2f}% had at least one woman and one LMIC-based researcher among the authors.")


Combined Among the 4223 papers with non-missing gender and LMIC status information, 4.95% had at least one woman and one LMIC-based researcher among the authors.


In [ ]:
# Count the number of unique pubs in both dataframes
num_common_pubs = len(set(MIMIC_affli['pub_id']).intersection(Control_affli['pub_id']))

print("Number of common pubs: ", num_common_pubs)


Number of common pubs:  111


In [ ]:

num_common_pubs = len(set(MIMIC_pubs['id']).intersection(Control_pubs['id']))

print("Number of common pubs: ", num_common_pubs)


Number of common pubs:  112


In [ ]:
# Calculate the difference between the two groups
percentage_difference = (mimic_proportion - control_proportion) * 100

print(f"The difference between the two groups is {percentage_difference:.2f} percentage points.")


The difference between the two groups is 1.65 percentage points.


### Sensitivity Analysis

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
def sensitivity_analysis(total_treatment_papers, known_treatment_papers, treatment_counts, control_counts, num_scenarios=10):
    # Calculate the proportion of LMIC authors in the known data for each group
    treatment_proportion = treatment_counts[0] / sum(treatment_counts)
    control_proportion = control_counts[0] / sum(control_counts)

    # Calculate the number of papers with missing author information for each group
    missing_treatment_papers = total_treatment_papers - known_treatment_papers
    missing_control_papers = total_control_papers - known_control_papers
    missing_papers = missing_treatment_papers + missing_control_papers

    p_values = []

    for i in range(num_scenarios + 1):
        # Vary the proportion of LMIC authors in the missing data
        treatment_missing = int(missing_papers * treatment_proportion * i / num_scenarios)
        control_missing = int(missing_papers * control_proportion * i / num_scenarios)

        # Calculate the adjusted counts of LMIC authors for each group
        adjusted_treatment_counts = treatment_counts[0] + treatment_missing
        adjusted_control_counts = control_counts[0] + control_missing

        # Perform the one-sided Z-test with the alternative hypothesis stating that the proportion in the MIMIC/eICU group is greater than the control group
        z_stat, p_value = proportions_ztest([adjusted_treatment_counts, adjusted_control_counts],
                                            [known_treatment_papers + missing_treatment_papers, known_control_papers + missing_control_papers],
                                            alternative='larger')

        p_values.append(p_value)

    return p_values


In [ ]:
MIMIC_affli.pub_id.nunique()
# Control_pubs.id.nunique()

1672

In [ ]:
total_treatment_papers = MIMIC_pubs.id.nunique()
total_control_papers = Control_pubs.id.nunique()
known_treatment_papers = MIMIC_affli.pub_id.nunique()
known_control_papers = Control_affli.pub_id.nunique()
n_MIMIC_lmic = len(MIMIC_lmic_count)
n_Control_lmic = len(Control_lmic_count)
treatment_counts = (n_MIMIC_lmic, known_treatment_papers - n_MIMIC_lmic)
control_counts = (n_Control_lmic, known_treatment_papers - n_Control_lmic)

p_values = sensitivity_analysis(total_treatment_papers, known_treatment_papers,
                                treatment_counts, control_counts)

for i, p_value in enumerate(p_values):
    print(f"Scenario {i}: p-value = {p_value:.4f}")


Scenario 0: p-value = 0.2277
Scenario 1: p-value = 0.2231
Scenario 2: p-value = 0.2187
Scenario 3: p-value = 0.2143
Scenario 4: p-value = 0.2071
Scenario 5: p-value = 0.2031
Scenario 6: p-value = 0.1991
Scenario 7: p-value = 0.2080
Scenario 8: p-value = 0.1888
Scenario 9: p-value = 0.1851
Scenario 10: p-value = 0.1816


In [ ]:
#  Set data
# Replace these values with your data
total_treatment_papers = len(MIMIC_pubs)
total_control_papers = len(Control_pubs)
known_treatment_papers = MIMIC_affli.pub_id.nunique()
known_control_papers = Control_affli.pub_id.nunique()
first_treatment_counts = (num_MIMIC_lmic_first_authors, known_treatment_papers - num_MIMIC_lmic_first_authors)
first_control_counts = (num_Control_lmic_first_authors, known_control_papers - num_Control_lmic_first_authors)
last_treatment_counts = (num_MIMIC_lmic_last_authors, known_treatment_papers - num_MIMIC_lmic_last_authors)
last_control_counts = (num_Control_lmic_last_authors , known_control_papers - num_Control_lmic_last_authors)
n_MIMIC_lmic = len(MIMIC_lmic_count)
n_Control_lmic = len(Control_lmic_count)
treatment_counts = (n_MIMIC_lmic, known_treatment_papers - n_MIMIC_lmic)
control_counts = (n_Control_lmic, known_treatment_papers - n_Control_lmic)


In [ ]:
# from statsmodels.stats.proportion import proportions_ztest

def sensitivity_analysis_all_missing_lmic(total_treatment_papers, known_treatment_papers, treatment_counts, control_counts):
    # Calculate the number of papers with missing author information for each group
    missing_treatment_papers = total_treatment_papers - known_treatment_papers
    missing_control_papers = total_control_papers - known_control_papers

    # Assume all the missing authors are from LMIC and add them to the known LMIC author counts
    adjusted_treatment_counts = treatment_counts[0] + missing_treatment_papers
    adjusted_control_counts = control_counts[0] + missing_control_papers
    print("adjusted treatment  count : ",adjusted_treatment_counts)
    print("adjusted treatment proportion %: ", adjusted_treatment_counts/total_treatment_papers*100)
    print("adjusted control count % :", adjusted_control_counts)
    print("adjusted control proportion: ", adjusted_control_counts/total_treatment_papers*100)
    # Perform the one-sided Z-test with the alternative hypothesis stating that the proportion in the treatment group is greater than the control group
    z_stat, p_value = proportions_ztest([adjusted_treatment_counts, adjusted_control_counts],
                                        [total_treatment_papers, total_control_papers],
                                        alternative='larger')

    return p_value, z_stat

overall_p_value,overall_z_stat = sensitivity_analysis_all_missing_lmic(total_treatment_papers, known_treatment_papers,
                                                treatment_counts, control_counts)

first_p_value , first_z_stat= sensitivity_analysis_all_missing_lmic(total_treatment_papers, known_treatment_papers,
                                                first_treatment_counts, first_control_counts)
last_p_value, last_z_stat = sensitivity_analysis_all_missing_lmic(total_treatment_papers, known_treatment_papers,
                                                last_treatment_counts, last_control_counts)

print(f"overall All missing authors from LMIC sensitivity analysis: p-value = {overall_p_value:.10f}")
print(f"first author All missing authors from LMIC sensitivity analysis: p-value = {first_p_value:.10f}")
print(f"last author All missing authors from LMIC sensitivity analysis: p-value = {last_p_value:.10f}")
print("overall_z:", overall_z_stat)
print("first_z:", first_z_stat)
print("last_z:", last_z_stat)

adjusted treatment  count :  791
adjusted treatment proportion %:  34.28695275249242
adjusted control count % : 217
adjusted control proportion:  9.4061551798873
adjusted treatment  count :  742
adjusted treatment proportion %:  32.16298222800174
adjusted control count % : 157
adjusted control proportion:  6.805374945817079
adjusted treatment  count :  741
adjusted treatment proportion %:  32.11963589076723
adjusted control count % : 161
adjusted control proportion:  6.978760294755093
overall All missing authors from LMIC sensitivity analysis: p-value = 0.0000000000
first author All missing authors from LMIC sensitivity analysis: p-value = 0.0000000000
last author All missing authors from LMIC sensitivity analysis: p-value = 0.0000000000
overall_z: 24.38890624121113
first_z: 25.436894931166147
last_z: 25.231906916459092


In [ ]:
def sensitivity_analysis_impute(total_treatment_papers, known_treatment_papers, treatment_counts, control_counts):
    # Calculate the proportion of LMIC authors in the known data for each group
    treatment_proportion = treatment_counts[0] / sum(treatment_counts)
    control_proportion = control_counts[0] / sum(control_counts)

    # Calculate the number of papers with missing author information for each group
    missing_treatment_papers = total_treatment_papers - known_treatment_papers
    missing_control_papers = total_control_papers - known_control_papers

    # Impute the missing data by distributing the LMIC and non-LMIC authors based on the known proportions for both groups
    imputed_treatment_counts = int(missing_treatment_papers * treatment_proportion)
    imputed_control_counts = int(missing_control_papers * control_proportion)

    # Calculate the adjusted counts of LMIC authors for each group
    adjusted_treatment_counts = treatment_counts[0] + imputed_treatment_counts
    adjusted_control_counts = control_counts[0] + imputed_control_counts
    print("adjusted treatment  count : ",adjusted_treatment_counts)
    print("adjusted treatment proportion %: ", adjusted_treatment_counts/total_treatment_papers*100)
    print("adjusted control count % :", adjusted_control_counts)
    print("adjusted control proportion: ", adjusted_control_counts/total_treatment_papers*100)
    # Perform the one-sided Z-test with the alternative hypothesis stating that the proportion in the treatment group is greater than the control group
    z_stat, p_value = proportions_ztest([adjusted_treatment_counts, adjusted_control_counts],
                                        [total_treatment_papers, total_control_papers],
                                        alternative='larger')

    return p_value

overall_p_value = sensitivity_analysis_impute(total_treatment_papers, known_treatment_papers,
                                                treatment_counts, control_counts)
first_p_value = sensitivity_analysis_impute(total_treatment_papers, known_treatment_papers,
                                                first_treatment_counts, first_control_counts)
last_p_value = sensitivity_analysis_impute(total_treatment_papers, known_treatment_papers,
                                                last_treatment_counts, last_control_counts)

print(f"impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = {overall_p_value:.5f}")
print(f"first author impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = {first_p_value:.5f}")
print(f"last author impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = {last_p_value:.5f}")



adjusted treatment  count :  215
adjusted treatment proportion %:  9.319462505418292
adjusted control count % : 185
adjusted control proportion:  8.019072388383181
adjusted treatment  count :  147
adjusted treatment proportion %:  6.371911573472041
adjusted control count % : 123
adjusted control proportion:  5.331599479843954
adjusted treatment  count :  146
adjusted treatment proportion %:  6.328565236237537
adjusted control count % : 127
adjusted control proportion:  5.504984828781968
impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = 0.00003
first author impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = 0.00025
last author impute missing from distribution of auhtors from LMIC sensitivity analysis: p-value = 0.00076


In [ ]:

def sensitivity_analysis_all_missing_not_lmic(total_treatment_papers, known_treatment_papers, treatment_counts, control_counts):
    # Since all missing authors are assumed to be not from LMIC, use the known LMIC author counts as they are
    adjusted_treatment_counts = treatment_counts[0]
    adjusted_control_counts = control_counts[0]
    print("adjusted treatment  count : ",adjusted_treatment_counts)
    print("adjusted treatment proportion %: ", adjusted_treatment_counts/total_treatment_papers*100)
    print("adjusted control count % :", adjusted_control_counts)
    print("adjusted control proportion: ", adjusted_control_counts/total_treatment_papers*100)
    # Perform the one-sided Z-test with the alternative hypothesis stating that the proportion in the treatment group is greater than the control group
    z_stat, p_value = proportions_ztest([adjusted_treatment_counts, adjusted_control_counts],
                                        [total_treatment_papers, total_control_papers],
                                        alternative='larger')

    return p_value

overall_p_value = sensitivity_analysis_all_missing_not_lmic(total_treatment_papers, known_treatment_papers,
                                                treatment_counts, control_counts)
first_p_value = sensitivity_analysis_all_missing_not_lmic(total_treatment_papers, known_treatment_papers,
                                                first_treatment_counts, first_control_counts)
last_p_value = sensitivity_analysis_all_missing_not_lmic(total_treatment_papers, known_treatment_papers,
                                                last_treatment_counts, last_control_counts)

print(f"overall All missing authors not from LMIC sensitivity analysis: p-value = {overall_p_value:.5f}")
print(f"first author All missing authors not from LMIC sensitivity analysis: p-value = {first_p_value:.5f}")
print(f"last author All missing authors not from LMIC sensitivity analysis: p-value = {last_p_value:.5f}")

adjusted treatment  count :  156
adjusted treatment proportion %:  6.762028608582575
adjusted control count % : 182
adjusted control proportion:  7.88903337667967
adjusted treatment  count :  107
adjusted treatment proportion %:  4.638058084091894
adjusted control count % : 122
adjusted control proportion:  5.28825314260945
adjusted treatment  count :  106
adjusted treatment proportion %:  4.59471174685739
adjusted control count % : 126
adjusted control proportion:  5.461638491547465
overall All missing authors not from LMIC sensitivity analysis: p-value = 0.22770
first author All missing authors not from LMIC sensitivity analysis: p-value = 0.21605
last author All missing authors not from LMIC sensitivity analysis: p-value = 0.32055


## Intersection of gender and LMIC

In [ ]:
print(MIMIC_pubs.columns)
print(MIMIC_affli.columns)


Index(['Unnamed: 0', 'id', 'title', 'abstract', 'authors', 'authors_count',
       'category_for', 'category_hra', 'category_hrcs_hc', 'category_hrcs_rac',
       'category_rcdc', 'category_sdg', 'category_uoa', 'concepts_scores',
       'doi', 'open_access', 'publisher', 'times_cited', 'type', 'year',
       'journal.id', 'journal.title', 'editors', 'funder_countries', 'funders',
       'mesh_terms', 'pages', 'pmid', 'research_org_cities',
       'research_org_countries', 'research_org_country_names',
       'research_org_names', 'research_orgs', 'researchers', 'volume', 'issue',
       'supporting_grant_ids', 'category_bra', 'research_org_state_names',
       'research_org_state_codes', 'category_icrp_ct', 'category_icrp_cso',
       'field_citation_ratio', 'relative_citation_ratio',
       'author_count_categories', 'first_names', 'genders', 'num_female',
       'num_male', 'num_na', 'pct_female_real', 'female_first', 'female_last',
       'pct_female_pessimistic', 'pct_female_optim

In [ ]:

# Merge the DataFrames
merged_df = pd.merge(MIMIC_affli, MIMIC_pubs, left_on='pub_id', right_on='id', how='left')

# Filter the dataset
filtered_df = merged_df.dropna(subset=['pct_female_real', 'LMIC'])

# Count papers with at least one woman (using pct_female_real >= 0.5) and one LMIC-based researcher among the authors
grouped = filtered_df.groupby('pub_id').agg({'pct_female_real': lambda x: (x >= 0.5).any(), 'LMIC': lambda x: (x == 1).any()})
papers_with_woman_and_lmic = grouped[(grouped['pct_female_real']) & (grouped['LMIC'])].shape[0]

# Calculate the percentage
total_papers = filtered_df['pub_id'].nunique()
percentage = (papers_with_woman_and_lmic / total_papers) * 100

print(f"Percentage of MIMIC/eICU papers with at least one woman and one LMIC-based researcher among the authors: {percentage:.2f}%")


Percentage of MIMIC/eICU papers with at least one woman and one LMIC-based researcher among the authors: 2.74%


In [ ]:
merged_df.gender

0          male
1        female
2        female
3        female
4          male
          ...  
13879      male
13880      male
13881      male
13882      male
13883      male
Name: gender, Length: 13884, dtype: object

In [ ]:
# Merge the DataFrames
merged_df = pd.merge(MIMIC_affli, MIMIC_pubs, left_on='pub_id', right_on='id', how='left')

# Filter the dataset
filtered_df = merged_df.dropna(subset=['gender', 'LMIC'])

# Count papers with at least one woman (using gender == 'F') and one LMIC-based researcher among the authors
grouped = filtered_df.groupby('pub_id').agg({'gender': lambda x: (x == 'female').any(), 'LMIC': lambda x: (x == 'Yes').any()})
papers_with_woman_and_lmic = grouped[(grouped['gender']) & (grouped['LMIC'])].shape[0]

# Calculate the percentage
total_papers = filtered_df['pub_id'].nunique()
percentage = (papers_with_woman_and_lmic / total_papers) * 100

print(f"Percentage of MIMIC/eICU papers with at least one woman and one LMIC-based researcher among the authors: {percentage:.2f}%")


Percentage of MIMIC/eICU papers with at least one woman and one LMIC-based researcher among the authors: 0.00%


## Check Author

In [ ]:
MIMIC_auth.researcher_id.nunique()

6606

In [ ]:
MIMIC_affli.researcher_id.unique()

array(['ur.014116573320.23', 'ur.016241001520.36', 'ur.013553641416.27',
       ..., 'ur.013705635017.95', 'ur.0743377731.49',
       'ur.012014630262.69'], dtype=object)

In [ ]:
MIMIC_affli.pub_id.nunique()

1672

In [ ]:
Control_affli.researcher_id.nunique()

14270